In [8]:
# ord : 문자의 유니코드 값을 돌려주는 함수
# chr : 유니코드 값을 입력받아 그 코드에 해당하는 문자를 출력하는 함수
print('ord :', ord('ㅎ'))
print('chr :', chr(12622))

ord : 12622
chr : ㅎ


In [9]:
-5%3

1

In [23]:
# string을 integer로 바꿔준는 문자열 hashing
def ploynomialRollingHash(str): 
    p = 31
    m = 1e9 + 9    # 10^9 + 9
    power_of_p = 1
    hash_val = 0
    
    for i in range(len(str)):
        hash_val = ((hash_val + (ord(str[i])-ord('a')+1)*power_of_p)%m)
        power_of_p = (power_of_p*p)%m
        
    return int(hash_val)

In [24]:
str1 = "geeksforgeeks"
print("Hash of '{}' = {}".format(str1, polynomialRollingHash(str1)))

Hash of 'geeksforgeeks' = 111226362


In [28]:
# 문자열을 길이가 k만큼 분할하기!
temp = 'convert'
k = 5

for i in range(len(temp)-k+1):
    print('{} ~ {} : {}'.format(i, i+k, temp[i:i+k]))

0 ~ 5 : conve
1 ~ 6 : onver
2 ~ 7 : nvert


In [31]:
import os

path = "./"
file_list = os.listdir(path)   # 현재 path에 있는 모든 파일 리스트를 가져온다

print("file list : {}".format(file_list))

file list : ['.ipynb_checkpoints', '211227.ipynb', 'IMDb_Reviews.csv', 'Kernel 연결된거 처음임.ipynb', 'LSH.ipynb', 'test_data.csv', 'train_data.csv', '설치된 lib.ipynb']


# Locality-Sensitive Hashing

In [1]:
class LSH:
    def __init__(self):
        pass
    
    # 문자열 해싱 : ploynomial rolling hash function
    def string_hashing(self, text):
        p = np = 53
        m = 10**9 + 7
        ret = 0
        
        for c in text:
            ret += np*ord(c)
            np *= p
            ret %= m
        
        return ret
    
    
    def ploynomial_hashing(self, a, b, x):   # a.b : 랜덤값, x : 문자열 해싱값
        p = 10**9 + 7
        return (a*x + b)%p
    
    
    def get_shingles(self, k, path):
        """ 기존 code
        # path에 있는 file 열기
        file = open(path, 'r', encoding='utf8')
        plain = file.read()
        
        # 길이 k씩 문자열 자름 
        sg = [plain[i:i+k] for i in range(len(plain) - k + 1)]
        file.close()
        
        # set : 집합 자료형 생성 (중복 허용 X, 순서가 없다!)
        ret = list(set(sg))
        """
        
        with open(path, 'r', encoding='utf-8') as f:
            plain = f.read()   # file의 내용 전체를 문자열로 돌려줌
            sg = [plain[i:i+k] for i in range(len(plain)-k+1)]
        ret = list(set(sg))
        return ret
    
    
    def min_hashing(self, shingles, a, b):
        ret, ch = 0, False
        
        # shingle을 조회하면서 hashing 값이 가장 큰 값 추출
        for shingle in shingles:
            h = self.string_hashing(shingle)
            sig = self.ploynomial_hashing(a, b, h)
            
            if not ch:
                ret = sig
                ch = True
            elif ret > sig:
                ret = sig
        
        return ret
    
    
    def get_signature(self, shingles, rand_hashes):   # signature 
        signatures = []
        for r in rand_hashes:
            signatures.append(self.min_hashing(shingles, r[0], r[1]))
        return signatures
    
    
    def run_LSH(self, sig_mat, b, r):
        # key: bucket의 키값, value : 해당 bucket에 속한 원소를 저장하는 집합
        buckets = dict()
        
        for t in range(len(sig_mat)):
            sig = sig_mat[t]
            
            # band 수만큼 반복 
            for i in range(b):
                band = ''.join([str(sig[j]) for j in range(i*r, (i+1)*r)])
                if not band in buckets:
                    buckets[band] = set()
                buckets[band].add(t)
        return buckets

In [2]:
import os
import random

lsh = LSH()

b,r = 100, 5
rand_hashes = [[random.randint(0, 10**9), random.randint(0, 10**9)] for i in range(b*r)]
print('rand_hashes :', len(rand_hashes), 'x', len(rand_hashes[0]))

s = (1/b)**(1/r)
print('s =', s)

rand_hashes : 500 x 2
s = 0.39810717055349726


In [3]:
path = './LSH_TEST/'
files = os.listdir(path)

all_shingles = []
sig_mat = []
fname = []

# 파일마다 조회
for f in files:
    fp = path + f
    if os.path.isdir(fp):  # 디렉토리면 pass
        continue    
    fname.append(f)
    
    # k = 10인 shingle 생성하기
    # 1단계 shingles
    shingles = lsh.get_shingles(10, fp)   
    all_shingles.append(set(shingles))
    print('get shingles complete', len(shingles))
    
    # 2단계 min-hashing
    signatures = lsh.get_signature(shingles, rand_hashes)
    print(fp, ":", signatures)
    sig_mat.append(signatures)

# 3단계 LSH
buckets = lsh.run_LSH(sig_mat, b, r)

get shingles complete 1191
./LSH_TEST/all i want for christmas.txt : [413395, 142403, 103116, 419017, 200153, 310788, 431811, 791092, 1269160, 39143, 500796, 2442647, 195689, 927771, 943559, 1255256, 824620, 16983, 613479, 50787, 60720, 12657, 265247, 1849900, 520414, 418708, 1529918, 1002554, 155151, 2213740, 342788, 518439, 2077449, 1349798, 1439995, 692506, 93467, 1617853, 267985, 86632, 1291072, 381976, 1169143, 1037440, 804097, 1467945, 212479, 832040, 360892, 3665, 1115294, 241162, 2632043, 2715214, 91295, 1219284, 21753, 727914, 182648, 1780615, 669597, 780851, 243285, 404366, 792363, 593971, 273807, 1017469, 1156506, 175748, 12077, 1964233, 491732, 1529192, 3005168, 363022, 1139701, 372140, 591730, 1253259, 2372677, 333105, 1439445, 111853, 667665, 1908109, 51970, 1405119, 1400634, 360907, 471480, 881772, 952935, 1983122, 1973246, 297091, 1242643, 275242, 1717608, 294646, 1298793, 832467, 1397154, 536208, 1111165, 742898, 943038, 572083, 1004194, 773936, 162006, 215290, 25327, 

./LSH_TEST/sample2.txt : [413395, 142403, 103116, 419017, 200153, 310788, 431811, 791092, 1269160, 39143, 500796, 2442647, 195689, 927771, 943559, 1255256, 824620, 16983, 613479, 50787, 60720, 12657, 265247, 1849900, 520414, 418708, 1529918, 1002554, 155151, 2213740, 342788, 518439, 2077449, 1349798, 1439995, 692506, 93467, 1617853, 267985, 86632, 1291072, 381976, 1169143, 1037440, 804097, 1467945, 212479, 832040, 360892, 3665, 1115294, 241162, 2632043, 2715214, 91295, 1219284, 21753, 727914, 182648, 1780615, 669597, 780851, 243285, 404366, 792363, 593971, 273807, 1017469, 1156506, 175748, 12077, 1964233, 491732, 1529192, 3005168, 363022, 1139701, 372140, 591730, 1253259, 2372677, 333105, 1439445, 111853, 667665, 1908109, 51970, 1405119, 1400634, 360907, 471480, 881772, 952935, 1983122, 1973246, 297091, 1242643, 275242, 1717608, 294646, 1298793, 832467, 1397154, 536208, 1111165, 742898, 943038, 572083, 1004194, 773936, 162006, 215290, 25327, 818544, 672886, 919960, 1895790, 1265658, 11

In [4]:
print('===== all shingles ===== ')
print('총 갯수 :', len(all_shingles))
for i in range(len(all_shingles)):
    print('{}번째 갯수 : {}'.format(i, len(all_shingles[i])))

===== all shingles ===== 
총 갯수 : 4
0번째 갯수 : 1191
1번째 갯수 : 529
2번째 갯수 : 1198
3번째 갯수 : 1025


In [5]:
print('signature matrix : ', len(sig_mat), 'x', len(sig_mat[0]))

signature matrix :  4 x 500


In [10]:
simfiles

[{2, 3}, set(), {0, 3}, {0, 2}]

In [15]:
# 파일 갯수만큼 empty 집합 생성 (※ 집합이므로 중복된 파일의 id가 없음
simfiles = [set() for i in range(len(fname))]

# 모든 버킷 출력
for bucket in buckets:
    print('bucket {}: '.format(bucket), end='')
    tmp_bucket = []  # bucket 안에 있는 파일들의 id(0~len(fname)-1)를 저장하는 리스트
    for fidx in buckets[bucket]:
        tmp_bucket.append(fidx)
        print(fname[fidx], end=', ')
        
    # 한 버킷에 속한 파일 처리
    for j in range(len(tmp_bucket)):
        for i in range(j+1, len(tmp_bucket)):
            simfiles[tmp_bucket[j]].add(tmp_bucket[i])
            simfiles[tmp_bucket[i]].add(tmp_bucket[j])
    print()

bucket 413395142403103116419017200153: all i want for christmas.txt, sample2.txt, sample3.txt, 
bucket 310788431811791092126916039143: all i want for christmas.txt, sample2.txt, sample3.txt, 
bucket 5007962442647195689927771943559: all i want for christmas.txt, sample2.txt, 
bucket 12552568246201698361347950787: all i want for christmas.txt, sample2.txt, 
bucket 60720126572652471849900520414: all i want for christmas.txt, sample2.txt, sample3.txt, 
bucket 418708152991810025541551512213740: all i want for christmas.txt, sample2.txt, sample3.txt, 
bucket 342788518439207744913497981439995: all i want for christmas.txt, sample2.txt, sample3.txt, 
bucket 69250693467161785326798586632: all i want for christmas.txt, sample2.txt, sample3.txt, 
bucket 129107238197611691431037440804097: all i want for christmas.txt, sample2.txt, 
bucket 14679452124798320403608923665: all i want for christmas.txt, sample2.txt, sample3.txt, 
bucket 11152942411622632043271521491295: all i want for christmas.txt, sa

In [17]:
# 각 파일 별로 비슷한 파일 출력 (같은 bucket에 한번이라도 존재한 경우)
print('similar files according to LSH (s={})'.format(s))
for j in range(len(fname)):
    print(fname[j] + "와 비슷한 파일들 : ", end='')
    for f in simfiles[j]:
        print(fname[f] + ", ", end='')
    print()

similar files according to LSH (s=0.39810717055349726)
all i want for christmas.txt와 비슷한 파일들 : sample2.txt, sample3.txt, 
jingle bells.txt와 비슷한 파일들 : 
sample2.txt와 비슷한 파일들 : all i want for christmas.txt, sample3.txt, 
sample3.txt와 비슷한 파일들 : all i want for christmas.txt, sample2.txt, 


* sample2 : all i want for christmas 끝에 1줄 제거
* sample3 : all i want for christmas 중간에 5줄 제거

In [18]:
print('shingles에 대한 실제 유사도 계산 값')
for i in range(len(all_shingles)):
    for j in range(i+1, len(all_shingles)):
        sim = len(all_shingles[i] & all_shingles[j]) / len(all_shingles[i] | all_shingles[j])
        print('sim({}, {}) = {}'.format(fname[i][:-4], fname[j][:-4], sim))

shingles에 대한 실제 유사도 계산 값
sim(all i want for christmas, jingle bells) = 0.0
sim(all i want for christmas, sample2) = 0.9941569282136895
sim(all i want for christmas, sample3) = 0.8512949039264829
sim(jingle bells, sample2) = 0.0
sim(jingle bells, sample3) = 0.0
sim(sample2, sample3) = 0.8463455149501661
